In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from glob import glob
from datetime import datetime

In [2]:
DATA_PATH = "./data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "char_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
question_data = pd.read_csv(QUEST_PATH)
word_embedding_data = pd.read_csv(WORD_EMBED_PATH, delimiter=" ", header=None, index_col=0)
char_embedding_data = pd.read_csv(CHAR_EMBED_PATH, delimiter=" ", header=None, index_col=0)

question_data["words"] = question_data["words"].str.split(" ")
question_data["chars"] = question_data["chars"].str.split(" ")

label = train_data["label"].values

In [3]:
from keras.preprocessing.text import Tokenizer

MAX_COUNT = 10000

word_tokenizer = Tokenizer(MAX_COUNT)
word_tokenizer.fit_on_texts(question_data["words"])

word_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, word_embedding_data.shape[1]), dtype=np.float64),
        word_embedding_data.loc[list(word_tokenizer.word_index.keys())[:MAX_COUNT]].values
    ),
    axis=0
)

char_tokenizer = Tokenizer(MAX_COUNT)
char_tokenizer.fit_on_texts(question_data["chars"])

char_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, char_embedding_data.shape[1]), dtype=np.float64),
        char_embedding_data.loc[list(char_tokenizer.word_index.keys())[:MAX_COUNT]].values
    ),
    axis=0
)


word_embedding_data.shape, char_embedding_data.shape

Using TensorFlow backend.


((10001, 300), (3049, 300))

In [4]:
from keras.preprocessing.sequence import pad_sequences

SEQ_LEN = 30

def gen_word_data(data):
    seq_word1 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["words"])
    seq_word2 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["words"])
    return pad_sequences(seq_word1, maxlen=SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_word2, maxlen=SEQ_LEN, padding="pre",truncating="pre")
    
def gen_char_data(data):
    seq_char1 = char_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["chars"])
    seq_char2 = char_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["chars"])
    return pad_sequences(seq_char1, maxlen=SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_char2, maxlen=SEQ_LEN, padding="pre",truncating="pre")

word1, word2 = gen_word_data(train_data)
char1, char2 = gen_char_data(train_data)
test_word1, test_word2 = gen_word_data(test_data)
test_char1, test_char2 = gen_char_data(test_data)

word1.shape, word2.shape, test_word1.shape, test_word2.shape, char1.shape, char2.shape, test_char1.shape, test_char2.shape

((254386, 30),
 (254386, 30),
 (172956, 30),
 (172956, 30),
 (254386, 30),
 (254386, 30),
 (172956, 30),
 (172956, 30))

In [5]:
from keras.models import Model
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM, Bidirectional, TimeDistributed
from keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dropout, BatchNormalization, Dense, Flatten, Lambda, K

In [6]:
# general
NUM_EPOCHES = 50
BATCH_SIZE = 1024
DENSE_SIZE = 300 # 512
DROP_RATE = 0.3

# cnn
CONV_LEN_1 = 128
CONV_LEN_2 = 128
CONV_LEN_3 = 128
CONV_LEN_4 = 128
CONV_LEN_5 = 128
CONV_LEN_6 = 128
CONV_LEN = CONV_LEN_1 + CONV_LEN_2 + CONV_LEN_3 + CONV_LEN_4 + CONV_LEN_5 + CONV_LEN_6

# lstm
LSTM_SIZE_1 = 256
LSTM_SIZE_2 = 256
DROP_RATE_LSTM = 0.3

In [ ]:
def cnn_layer_1(input1, input2, kernel_size, filters):
    conv = Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu")
    
    conv_a = conv(input1)
    conv_a = GlobalAveragePooling1D()(conv_a)
    
    conv_b = conv(input2)
    conv_b = GlobalAveragePooling1D()(conv_b)
    return conv_a, conv_b

In [ ]:
from sklearn.model_selection import StratifiedKFold

best_results = []
last_results = []

for i, (train_index, dev_index) in enumerate(StratifiedKFold(n_splits=10).split(X=char1, y=label)):
    train_char1, train_char2, train_y = char1[train_index, :], char2[train_index, :], label[train_index]
    dev_char1, dev_char2, dev_y = char1[dev_index, :], char2[dev_index, :], label[dev_index]
    
    input1 = Input(shape=(SEQ_LEN,), dtype="int32")
    input2 = Input(shape=(SEQ_LEN,), dtype="int32")

    embedding_layer = Embedding(
        input_dim=char_embedding_data.shape[0],
        output_dim=char_embedding_data.shape[1],
        weights=[char_embedding_data],
        input_length=SEQ_LEN,
        trainable=False
    )

    vector1 = embedding_layer(input1)
    vector2 = embedding_layer(input2)

    lstm_layer1 = LSTM(LSTM_SIZE_1, dropout=DROP_RATE, recurrent_dropout=DROP_RATE, return_sequences=True)
    first_1 = lstm_layer1(vector1)
    first_1 = Dropout(DROP_RATE)(first_1)
    first_2 = lstm_layer1(vector2)
    first_2 = Dropout(DROP_RATE)(first_2)

    lstm_layer2 = LSTM(LSTM_SIZE_2, dropout=DROP_RATE, recurrent_dropout=DROP_RATE, return_sequences=True)
    second_1 = lstm_layer2(first_1)
    second_2 = lstm_layer2(first_2)

    conv1a, conv1b = cnn_layer_1(second_1, second_2, kernel_size=1, filters=CONV_LEN_1)
    conv2a, conv2b = cnn_layer_1(second_1, second_2, kernel_size=2, filters=CONV_LEN_2)
    conv3a, conv3b = cnn_layer_1(second_1, second_2, kernel_size=3, filters=CONV_LEN_3)
    conv4a, conv4b = cnn_layer_1(second_1, second_2, kernel_size=4, filters=CONV_LEN_4)
    conv5a, conv5b = cnn_layer_1(second_1, second_2, kernel_size=5, filters=CONV_LEN_5)
    conv6a, conv6b = cnn_layer_1(second_1, second_2, kernel_size=6, filters=CONV_LEN_6)

    merge_a = concatenate([conv1a, conv2a, conv3a, conv4a, conv5a, conv6a])
    merge_b = concatenate([conv1b, conv2b, conv3b, conv4b, conv5b, conv6b])
    diff = Lambda(lambda x: K.abs(x[0] - x[1]))([merge_a, merge_b])
    mult = Lambda(lambda x: x[0] * x[1])([merge_a, merge_b])
    merge = concatenate([diff, mult])

    x = Dropout(DROP_RATE)(merge)
    x = BatchNormalization()(x)

    x = Dense(DENSE_SIZE, activation="relu")(x)
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)

    pred = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=[input1, input2], outputs=pred)
    model.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=["acc"]
    )

    early_stopping = EarlyStopping("val_loss", patience=10)
    check_point = ModelCheckpoint(
        "./log/%s.multi_lstm_cnn_char.{epoch:03d}.hdf5" % (datetime.now().strftime("%Y%m%d-%H%M%S")),
        monitor="val_loss",
        save_best_only=True,
    )

    fit_res = model.fit(
        x=[train_char1, train_char2],
        y=train_y,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHES,
        validation_data=([dev_char1, dev_char2], dev_y),
        shuffle=True,
        callbacks=[early_stopping, check_point]
    )

    pred_last = model.predict([test_char1, test_char2], batch_size=BATCH_SIZE)
    last_results.append(pd.DataFrame(pred_last, columns=["y_pre"]))
    

    print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
    model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))
    pred_best = model.predict([test_char1, test_char2], batch_size=BATCH_SIZE)
    best_results.append(pd.DataFrame(pred_best, columns=["y_pre"]))

pd.DataFrame(pd.concat(last_results, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-multi_lstm_cnn_char_last.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),
    index=False
)
pd.DataFrame(pd.concat(best_results, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-multi_lstm_cnn_char_best.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),
    index=False
)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 228946 samples, validate on 25440 samples
Epoch 1/50
228946/228946 [==============================] - 138s 603us/step - loss: 0.5012 - acc: 0.7589 - val_loss: 0.3626 - val_acc: 0.8330
Epoch 2/50
228946/228946 [==============================] - 133s 582us/step - loss: 0.3604 - acc: 0.8375 - val_loss: 0.3043 - val_acc: 0.8669
Epoch 3/50
228946/228946 [==============================] - 134s 583us/step - loss: 0.3070 - acc: 0.8653 - val_loss: 0.2745 - val_acc: 0.8809
Epoch 4/50
 54272/228946 [======>.......................] - ETA: 1:37 - loss: 0.2789 - acc: 0.8789